In [1]:
from operator import add

from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from typing_extensions import TypedDict
from typing import Annotated

In [2]:
class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]


def node_a(state: State) -> dict:
    return {"foo": "a", "bar": ["a"]}


def node_b(state: State) -> dict:
    return {"foo": "b", "bar": ["b"]}

In [3]:
workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)
checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

In [4]:
graph.invoke({"foo": ""}, {"configurable": {"thread_id": "1"}})

{'foo': 'b', 'bar': ['a', 'b']}

In [5]:
config = {"configurable": {"thread_id": "1"}}
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5152-d76e-65b6-8002-b5d1b8ca540d'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-19T06:58:39.623723+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5152-d769-6999-8001-d6343b3b2296'}}, tasks=(), interrupts=())

In [6]:
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5152-d76e-65b6-8002-b5d1b8ca540d'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-19T06:58:39.623723+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5152-d769-6999-8001-d6343b3b2296'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5152-d769-6999-8001-d6343b3b2296'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-19T06:58:39.621775+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c5152-d764-695a-8000-85bb4b696054'}}, tasks=(PregelTask(id='520e8448-7c43-762b-6316-9bcad9a064db', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts

In [12]:
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f0c50f1-a5f7-62a7-8001-8e8775e49131"}}
graph.invoke(None, config=config)

{'foo': 'b', 'bar': ['a', 'b']}

In [20]:
graph.get_state(config)

StateSnapshot(values={'foo': 1232, 'bar': ['a', 'b', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c4558-c80d-61b0-8003-17f3d3806630'}}, metadata={'source': 'update', 'step': 3, 'parents': {}}, created_at='2025-11-18T08:06:55.700216+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c449f-d4e0-6783-8002-cc72fdef74c9'}}, tasks=(), interrupts=())